In [48]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
import langchain
from langchain.prompts import load_prompt
from langchain.llms import OpenAIChat
from langchain import PromptTemplate, LLMChain, FewShotPromptTemplate
import re

In [3]:
#load the api key
openai.api_key = os.getenv('OPENAI_KEY')
#add the key to the openai api

TOKEN_USAGE = 0
COST_PER_TOKEN = 0.002 / 1000


In [4]:
def usage():
    global TOKEN_USAGE
    global COST_PER_TOKEN
    cost = COST_PER_TOKEN * TOKEN_USAGE
    print(f"Total token usage: {TOKEN_USAGE} | Cost: ${cost:.2f}")

In [5]:
#create an empty dataframe with a column for the prompt, the token cost and a column for the response
genrations_df = pd.DataFrame(columns=['prompt', 'token_cost', 'response', 'experiment_type'])

In [6]:
#function to generate a prompt, save the prompt, token cost and response to a dataframe and save the dataframe to a csv file
def generate_prompt(prompts, experiment_name):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=prompts
    )

    usage = response['usage']['total_tokens']
    global TOKEN_USAGE
    TOKEN_USAGE += usage
    
    text_response = response['choices'][0]['message']['content']
    #add the prompt, token cost and response to the dataframe and save it
    genrations_df.loc[len(genrations_df)] = [prompts, usage, text_response, experiment_name]
    #check if a directory for the experiment_name exists in generations folder, if not create it
    if not os.path.exists(f'generations/{experiment_name}'):
        os.makedirs(f'generations/{experiment_name}')
    genrations_df.to_csv(f'generations/generations_summary.csv')


    #create a new txt file for each prompt and response and save it to the experiments_name folder
    with open(f'generations/{experiment_name}/{len(genrations_df)-1}.txt', 'w') as f:
        f.write(f"Prompt: {prompts}\n")
        f.write(f"Response: {text_response}\n")
        f.write(f"Token usage: {usage}\n")
        f.write(f"Experiment type: {experiment_name}\n")
        f.write(f"Total token usage: {TOKEN_USAGE}\n") 

    return response['choices'][0]['message']['content']

In [48]:
print(generate_prompt(test_prompt, 'harness_setup'))
calculate_total_usage(COST_PER_TOKEN)

My programmers call me OpenAI, but you can call me whatever you like.
Total token usage: 292 | Cost: $0.00


In [7]:
class Prompt_Chain:
    def __init__(self, experiment_name, initial_prompt):
        self.experiment_name = experiment_name
        self.prompt = initial_prompt
    
    def user_says(self, user_input):
        self.prompt.append({ "role": "user", "content": user_input })
        return self

    def assistant_says(self, assistant_output):
        self.prompt.append({ "role": "assistant", "content": assistant_output })

    def wrap_text(self, text):
        text_to_print = ""
        if(len(text) > 200):
            for i in range(0, len(text), 200):
                text_to_print += text[i:i+200]
                text_to_print += " \n"
        print(text_to_print)
    
    def ask(self, user_text):
        self.user_says(user_text)
        response = generate_prompt(self.prompt, self.experiment_name)
        self.wrap_text(response)
        self.assistant_says(response)
    
    def refresh(self, initial_prompt):
        self.prompt = initial_prompt


### Attempts to setup a simple agent

In [124]:
prompt_chain = [
    {"role": "system", "content": "You are a sales agent for a pest control company. You are on the phone with a customer who is interested in setting up a service appointment. The customer is asking you about the different types of services you offer. Ask questions to understand their problem and then offer a solution."},
    {"role": "assistant", "content": "Hi, welcome to Better Termite. How can I help you today?"},
]

agent = Prompt_Chain('Agent Test', prompt_chain)

In [110]:
agent.ask("I have a problem with termites in my house. What can you do to help me?")

In [111]:
agent.ask("I've seen what looks like flying ants around my house. Are they termites?")

It's possible that those could be termites. We would need to do a thorough inspection to confirm whether or not you have a termite infestation. Would you like to schedule a free inspection with one of 
 our pest control experts? 



In [112]:
agent.ask("Does that cost anything?")

No, it's completely free. Our expert will come to your house and conduct a thorough examination of the property to determine if termites are present, and if so, the extent of the infestation. Once the 
y have completed their investigation, they will go over the findings with you and suggest a customized course of action based on your specific situation. Would you like to schedule an appointment for  
a free inspection? 



In [113]:
agent.ask("Yes please!")

Great! Let me take your information and schedule an appointment for you. Our termite experts will be out as soon as possible to take a look at the house and help you with a solution. In the meantime,  
I can let you know that we offer various termite treatment options based on the severity of the infestation. Our treatments include liquid treatment, fumigation, and baiting systems. Once the technici 
an has completed the inspection, they can provide you with a recommendation for the best treatment option for your situation. Would you like to learn more about any of these treatments now? 



In [84]:
agent.ask("My name is John Smith, and my phone number is 555-555-5555")

In [131]:
agent.refresh(prompt_chain)
usage()

Total token usage: 5468 | Cost: $0.01


In [126]:
agent.ask('I am interested in getting setup on a quarterly service plan. What do I need to do?')

Great! We offer several different service plans depending on your specific needs. Can you tell me a bit more about the pest problem you are experiencing? That way, I can recommend the best plan for yo 
u. 



In [127]:
agent.ask("It's more of a proactive thing. I have been seeing some ants around my house")

I understand. We have a quarterly service plan that is designed to prevent pests from entering your home. Our technicians will come to your home every quarter to spray the perimeter and any common are 
as where pests tend to enter. This service covers a wide range of pests like ants, spiders, roaches, silverfish, and more. Would you be interested in this plan? 



In [256]:
usage()

Total token usage: 8666 | Cost: $0.02


### Experimenting to Create My Own Agent

In [184]:
#contact information collected through form a the start of the conversation
first_name = "John"
last_name = "Smith"
phone_number = "555-555-5555"
email = 'georgeschulz33@gmail.com'
address = '4355 Penwood Drive'
city = 'Alexandria'
state = 'VA'
zip_code = '22310'


In [185]:
#agent screens geographic data

In [203]:
#agent attempts to build a customer profile of the issue, while exposing pain points
template = """ You are a sales agent for a pest control company. You are a chatbot texting with a propsect for a service program. Your goal is to get them to sign up for a service program. Respond by classifying the information as one of the following categories: Question, Contact, Special Instructions, or Customer Issue Detail. You should categorize all responses as one of the categories For example:

User: I have a problem with termites in my house. What can you do to help me?
Category: Customer Issue Detail

User: Do you offer green pest control services?
Category: Question

User: My name is John Smith, and my phone number is 555-555-5555
Category: Contact

User: There is a gate in the back of my house. Please make sure to close it when you leave.
Category: Special Instructions
"""

prompt_chain = [
    {"role": "system", "content": template},
    {"role": "assistant", "content": "Hi, welcome to Better Termite. How can I help you today?"},
]

agent = Prompt_Chain('Agent Test', prompt_chain)

In [204]:
print(agent.ask("This is G Schulz just reaching back out"))


None


In [205]:
agent.ask("I have a problem with termites in my house. What can you do to help me?")

Category: Customer Issue Detail

I'm sorry to hear that you have a termite problem. We offer a comprehensive termite treatment program that includes a combination of baiting and liquid application met 
hods. Our program is designed to eliminate existing termites and prevent future infestations. Would you like to learn more about this program? 



In [8]:
class TestSet:
    def __init__(self, test_set, function):
        self.test_set = test_set
        self.test_name = 'Untitled test'
        self.function = function
    
    async def run(self, name):
        self.test_name = name
        #iterate over test_set, a dictionary where each key is a dictionary
        for test in self.test_set:
            actual = await self.function(test)
            self.test_set[test]['actual'] = actual
            if self.test_set[test]['expected'] == actual:
                self.test_set[test]['result'] = 'pass'
            else:
                self.test_set[test]['result'] = 'fail'
        
        #calculate the overall percentage passed and failed
        total = len(self.test_set)
        passed = 0
        for test in self.test_set:
            if self.test_set[test]['result'] == 'pass':
                passed += 1

        print(f"Test {self.test_name} passed {passed} out of {total} tests")
        
        return self.test_set

    def check(self, key):
        value = self.test_set[key]['prompt']
        response = self.function(value)
        if response == self.test_set[key]['expected']:
            print("/ PASS | Result: " + response + " | Expected: " + self.test_set[key]['expected'])
        else:
            print("X FAIL | Result: " + response + " | Expected: " + self.test_set[key]['expected'])

#### A possible framework for thought in a coversation with an agent

1. Classify the type of request: 
2. If augmentation is required, adapt the prompt. If information is collected, add it to the customer profile. 
3. Adjust the conversational chain
4. Return the response

In [238]:
template = """Classify the response as one of the following categories: Question, Contact, Special Instructions, or Customer Issue Detail. Format as: Category: *category* Response: {customer_response}"""
prompt = PromptTemplate(template=template, input_variables=["customer_response"])
prompt.save('prompts/response_classification/short-1.json')

In [9]:
def classify_response_request_type(customer_response):
    prompt = load_prompt('prompts/response_classification/verbose-1.json')
    llm = OpenAIChat(openai_api_key=os.getenv('OPENAI_KEY'))
    llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="text")
    classification_raw = llm_chain.run(customer_response)
    #check if the terms: "Question", "Contact", "Special Instructions", or "Customer Issue Detail" are in the response. If they are, pull out the first that appears in classification_raw. Use a regex to pull out the category and get the
    match = re.search(r"(Question|Contact|Special Instructions|Customer Issue Detail)", classification_raw)
    return_value = "No Classification Found"
    if match:
        return_value = match.group(0)
    return return_value

Ways to categorize:

- Information Request: look up data
- Contact Information: store in contact profile
- Customer Issue Details: store in customer profile

Maybe it should be a boolean search with three tags?


In [ ]:

example_set = {
    "test1": {
        "prompt": "I have a problem with termites in my house. What can you do to help me?",
        "expected": "Customer Issue Detail"
    },
    "test2": {
        "prompt": "Do you offer green pest control services?",
        "expected": "Question"
    },
    "test3": {
        "prompt": "My name is John Smith, and my phone number is 555-555-5555",
        "expected": "Contact"
    },
}

test = TestSet(example_set, classify_response_request_type)
# test.check('test1')

Test Results: Poor performance probably do to overlapping categories and poorly defined categories. The next test will be to create a tagging series where there can be more than one classification. Each will be a boolean classsification prompt

In [49]:
contains_contact_info_prompt = """Determine whether the following response contains contact information for the person who sent the message. Write True if it contains contact info, and False if not. A response has contact information if it contains their name, phone, email or address info
Example:
Response: My name is John Smith, and my phone number is 555-555-5555
True
Response: I have a problem with termites in my house. What can you do to help me?
False
Response: I live in Bethesda
True
Response: My kid Andrian is allgeric to bees
False
Response: {customer_response}"""
prompt = PromptTemplate(template=contains_contact_info_prompt, input_variables=["customer_response"])



prompt.save('prompts/response_is_contact_info/contains_contact_info_example_interpolation.json')

In [50]:
def classify_contains_contact_info(customer_response):
    prompt = load_prompt('prompts/response_is_contact_info/contains_contact_info_2.json')
    llm = OpenAIChat(openai_api_key=os.getenv('OPENAI_KEY'))
    llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="text")
    classification_raw = llm_chain.run(customer_response)
    match = re.search(r"(True|False)", classification_raw)
    return_value = "No Classification Found"
    if match:
        return_value = match.group(0)
    return return_value

In [56]:
example_set = {
    "test1": {
        "prompt": "I have a problem with termites in my house. What can you do to help me?",
        "expected": "False"
    },
    "test2": {
        "prompt": "Do you offer green pest control services?",
        "expected": "False"
    },
    "test3": {
        "prompt": "My name is John Smith, and my phone number is 555-555-5555",
        "expected": "True"
    },
    "test4": {
        "prompt": "I live in Alexandria",
        "expected": "True"
    },
    "test5": {
        "prompt": "I am seeing them in the kitchen. They've been there for a few days now.",
        "expected": "False"
    },
    "test6": {
        "prompt": "Can you put this number on my account 5813213411",
        "expected": "True"
    }  
}

tests = TestSet(example_set, classify_contains_contact_info)
for test in tests.test_set:
     tests.check(test)


/ PASS | Result: False | Expected: False
/ PASS | Result: False | Expected: False
/ PASS | Result: True | Expected: True
/ PASS | Result: True | Expected: True
/ PASS | Result: False | Expected: False
X FAIL | Result: False | Expected: True


In [271]:
classify_contains_contact_info('My name is John Smith, and my phone number is 555-555-5555')

'True'

In [281]:
result

<coroutine object TestSet.run at 0x7f964b49b2c0>